In [2]:
# import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

In [3]:
from reader import *
from string_matching_by_word import *
from string_matching_by_spaCy_NP import *
from word_embeddings_by_spaCy_NP import *
from hobbs import entry
from nltk import pos_tag

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
file_number = 'c3'

### Read Document

In [6]:
input_path = '/Users/ambuj/Documents/MS Stuff/nlp_cs_6340/final_project/nlp-project2019/dev/{}.input'.format(file_number)
key_path = '/Users/ambuj/Documents/MS Stuff/nlp_cs_6340/final_project/nlp-project2019/dev/{}.key'.format(file_number)

In [7]:
read = ReadInput(input_path)
list_of_sentences = read.getListOfSentences()
full_text = read.getText()

### Get Sentence Dict and Cluster Head Dict

In [8]:
sentence_dict = getSentenceDict(list_of_sentences)
cluster_head_dict = getClusterHeads(sentence_dict)

### Get Coref for exact match by word

In [9]:
coref_dict_match_word = getCorefDict_match_word(sentence_dict, cluster_head_dict,90)
# coref_dict_match_word

### Get Coref for threshold match by NP

In [10]:
coref_dict_match_NP = getCorefDict_match_NP(sentence_dict, cluster_head_dict,80)
# coref_dict_match_NP

### Get Coref for Word Embedding Similarity by NP

In [11]:
# coref_dict_all_sorted = getCorefDict_all_sorted(sentence_dict, cluster_head_dict,0.5)
# coref_dict_max_sentence = getCorefDict_max_of_each_sentence(sentence_dict, cluster_head_dict,0.5)

In [12]:
# coref_dict_all_sorted_top3 = get_TopN_Matches(coref_dict_all_sorted,3)

### Merging dictionaries

In [28]:
def mergeDicts(dict1, dict2):
    
    d1 = dict1.copy()
    d2 = dict2.copy()
    
    d1_keys = d1.keys()
    d2_keys = d2.keys()
    
    for key in d1_keys:
        d1[key] = list(map(lambda x: x[0:3], d1[key]))
    
    for key in d2_keys:
        d2[key] = list(map(lambda x: x[0:3], d2[key]))
    
    for key in d2_keys:
        if(key not in d1_keys):
            d1[key] = d2[key]
        else:
            for val in d2[key]:
                if(val not in d1[key]):
                    d1[key].append(val)
    
    return dict(sorted(d1.items()))

In [29]:
coref_final = mergeDicts(coref_dict_match_word, coref_dict_match_NP)

In [30]:
coref_final

{'X1': [['outbreak', 8, 'outbreak'],
  ['outbreak', 9, 'outbreak'],
  ['outbreak', 10, 'outbreaks'],
  ['outbreak', 10, 'outbreaks'],
  ['outbreak', 11, 'outbreaks']],
 'X3': [[Western Berlin, 6, 'Berlin']],
 'X5': [[Department of Infectious Diseases, 3, 'our department'],
  [Department of Infectious Diseases, 5, 'the Department']],
 'X6': [['we', 3, 'we'], ['we', 5, 'we'], ['we', 11, 'We']],
 'X7': [['virus', 4, 'virus']]}

### Passing the reference dict to Hobbs

In [16]:
coref_final_with_pro = entry(list_of_sentences, cluster_head_dict, coref_final)

In [17]:
coref_final_with_pro

{'X1': [['outbreak', 8, 'outbreak'],
  ['outbreak', 9, 'outbreak'],
  ['outbreak', 10, 'outbreaks'],
  ['outbreak', 10, 'outbreaks'],
  ['outbreak', 11, 'outbreaks']],
 'X3': [[Western Berlin, 6, 'Berlin']],
 'X5': [[Department of Infectious Diseases, 3, 'our department'],
  [Department of Infectious Diseases, 5, 'the Department']],
 'X6': [['we', 3, 'we'], ['we', 5, 'we'], ['we', 11, 'We']],
 'X7': [['virus', 4, 'virus']],
 'X0': [['me', 8, 'Volkmer', 0], ['me', 10, 'Volkmer', 0]]}

### Adding those Cluster Heads which were not predicted

In [18]:
def addNOPredsClusterHeads(cluster_head_dict, coref_final_with_pro):
    for cluster_id, cluster_val in cluster_head_dict.items():
        if(cluster_id not in coref_final_with_pro.keys()):
            coref_final_with_pro[cluster_id] = [['',int(cluster_val[0]),'', None]]
            
    return coref_final_with_pro

In [19]:
coref_final_with_pro = addNOPredsClusterHeads(cluster_head_dict, coref_final_with_pro)

In [20]:
coref_final_with_pro

{'X1': [['outbreak', 8, 'outbreak'],
  ['outbreak', 9, 'outbreak'],
  ['outbreak', 10, 'outbreaks'],
  ['outbreak', 10, 'outbreaks'],
  ['outbreak', 11, 'outbreaks']],
 'X3': [[Western Berlin, 6, 'Berlin']],
 'X5': [[Department of Infectious Diseases, 3, 'our department'],
  [Department of Infectious Diseases, 5, 'the Department']],
 'X6': [['we', 3, 'we'], ['we', 5, 'we'], ['we', 11, 'We']],
 'X7': [['virus', 4, 'virus']],
 'X0': [['me', 8, 'Volkmer', 0], ['me', 10, 'Volkmer', 0]],
 'X2': [['', 0, '', None]],
 'X4': [['', 0, '', None]],
 'X8': [['', 6, '', None]],
 'X9': [['', 9, '', None]],
 'X10': [['', 12, '', None]],
 'X11': [['', 12, '', None]]}

### Taking only the head nouns

In [35]:
def takeHeadNouns(coref_final_with_pro):
    
    for cluster in coref_final_with_pro.keys():
        for i in range(0, len(coref_final_with_pro[cluster])):
            coref_final_with_pro[cluster][i][2] = coref_final_with_pro[cluster][i][2].split(' ')[-1]
                        
    return coref_final_with_pro

In [36]:
coref_final_with_pro = takeHeadNouns(coref_final_with_pro)

In [37]:
coref_final_with_pro

{'X1': [['outbreak', 8, 'outbreak'],
  ['outbreak', 9, 'outbreak'],
  ['outbreak', 10, 'outbreaks'],
  ['outbreak', 10, 'outbreaks'],
  ['outbreak', 11, 'outbreaks']],
 'X3': [[Western Berlin, 6, 'Berlin']],
 'X5': [[Department of Infectious Diseases, 3, 'department'],
  [Department of Infectious Diseases, 5, 'Department']],
 'X6': [['we', 3, 'we'], ['we', 5, 'we'], ['we', 11, 'We']],
 'X7': [['virus', 4, 'virus']],
 'X0': [['me', 8, 'Volkmer', 0], ['me', 10, 'Volkmer', 0]],
 'X2': [['', 0, '', None]],
 'X4': [['', 0, '', None]],
 'X8': [['', 6, '', None]],
 'X9': [['', 9, '', None]],
 'X10': [['', 12, '', None]],
 'X11': [['', 12, '', None]]}

### Print Output

In [38]:
def printOP(cluster_head_dict, coref_final_with_pro):
    
    for cluster_id, cluster_head_name in cluster_head_dict.items():

        print('<COREF ID="{}">{}</COREF>'.format(cluster_id, cluster_head_name[1]))

        coreferences = coref_final_with_pro[cluster_id]
        list_of_sent_ids = list(map(lambda x: x[1], coreferences))
        sorted_index_sent_ids = [i[0] for i in sorted(enumerate(list_of_sent_ids), key=lambda x:x[1])]
        coreferences = [coreferences[i] for i in sorted_index_sent_ids]

        for coref in coreferences:
            if(coref[0] == ''):
                continue
            print('{{{0}}}'.format(coref[1]) + ' ' + '{' + coref[2] + '}')
        print('\n', end = '')      

In [39]:
# %%capture cap --no-stderr
printOP(cluster_head_dict, coref_final_with_pro)

<COREF ID="X0">me</COREF>
{8} {Volkmer}
{10} {Volkmer}

<COREF ID="X1">outbreak</COREF>
{8} {outbreak}
{9} {outbreak}
{10} {outbreaks}
{10} {outbreaks}
{11} {outbreaks}

<COREF ID="X2">hundreds</COREF>

<COREF ID="X3">Western Berlin</COREF>
{6} {Berlin}

<COREF ID="X4">sixties</COREF>

<COREF ID="X5">Department of Infectious Diseases</COREF>
{3} {department}
{5} {Department}

<COREF ID="X6">we</COREF>
{3} {we}
{5} {we}
{11} {We}

<COREF ID="X7">virus</COREF>
{4} {virus}

<COREF ID="X8">brand</COREF>

<COREF ID="X9">occurrences</COREF>

<COREF ID="X10">Mods.MPP</COREF>

<COREF ID="X11">JW</COREF>



In [387]:
with open('{}.response'.format(file_number),'w') as f:
    f.write(cap.stdout)

### Answer Key

In [103]:
# reader_o = ReadInput(key_path)
# ans = reader_o.getListOfSentences()
# ans